The theme of this notebook is Returner.
Because the audience gets excited when the returner runs a long distance.
Me too. I've been a fan of the NFL since 1992. for example, Tom Rathman, Christian Okoye, John Taylor, Roger Craig, Bo Jackson,etc.. 
    
anyway, I'm a beginner in PYTHON. The training started on December 29th, and the following are the results of 5 days.
I will continue to study and challenge KAGGLE. In particular, I will do my best to automatically extract the features of human relationships using DGL and GCNN.


By the way, the feature of the returner that I paid attention to is the blocker player who runs in front of the returner.
I hypothesized that if the returner runs in hiding behind three blockers, he can run a longer distance invisible to the opposing team.

It's like Gundam's Black Tri-Stars.　you know?
Therefore, I want the track data to have an explanatory variable for the affinity between the three and the returner. For example, it is the affinity that I went to when I was taller and faster than the returner. By doing so, I thought that the relationship between the returner and the blocker could be connected and aggregated at the edge of the DGL to prove the hypothesis of the triple star.

In this analysis, there is not enough DGL edge data,
So I want to continue the analysis.
In unpredictable sports where diverse human resources continue to change, I think that we can build a stronger team by taking action on our own and embodying the ideal outline of the special team of the future.
    

Special thanks to NFL Big Data Bowl Basic EDA for beginner.
Special thanks to ”Win with Kaggle Data analysis technology” to all of authors and Gijutsu-Hyoron Co., Ltd..I referred to the sample code.



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

In [ ]:
plays = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')

#plays

In [ ]:
scores = plays['returnerId'] 
#plt.hist(scores.dropna(), cumulative=True, density=True, bins='auto')



In [ ]:
plays.describe()


In [ ]:
import plotly.express as px

#棒グラフを作成
fig = px.histogram(plays, x = 'returnerId')
#軸の設定等
#fig.update_layout(width=100,height=50,margin=dict(l=10,r=10,b=5,t=5),
#                 yaxis=dict(range=[0,10]),xaxis=dict(dtick=1, autorange=True),title=dict(font = dict(size=20)))

#データを並び替え
#fig = px.bar(scores)
fig.update_layout(xaxis={'categoryorder':'total descending'})
#グラフの表示
fig.show()

In [ ]:
plays.dropna(subset=['kickReturnYardage'])

plays = plays.drop(['playResult'], axis=1)

In [ ]:
PFFS = pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')

plays = pd.merge(plays, PFFS, on=['gameId', 'playId'])

In [ ]:
track2020 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')
track2020 = track2020.rename(columns={'nflId':'returnerId'})


In [ ]:
plays['returnerId'] = plays['returnerId'].str.replace(';','')
plays['returnerId'] = plays['returnerId'].astype(float)

In [ ]:
plays = pd.merge(plays, track2020, on=['gameId', 'playId','returnerId'], how='left')

In [ ]:
#print("plays", plays.shape)
plays.dtypes

In [ ]:
plays = plays.drop(['time','gameId','playId','playDescription','x','y'], axis=1)


In [ ]:
plays.dtypes

In [ ]:

train_x = plays.dropna(subset=['kickReturnYardage']).drop(['kickReturnYardage'], axis=1)
train_y = plays.dropna(subset=['kickReturnYardage'])['kickReturnYardage']

train_x




In [ ]:
#train_x.dtypes


In [ ]:
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 80)
train_x.describe(include='all')



In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

for c in ['possessionTeam', 'specialTeamsPlayType','specialTeamsResult','penaltyCodes','penaltyJerseyNumbers', 'yardlineSide', 'gameClock', 'passResult', 'snapDetail', 'kickType','kickDirectionIntended','kickDirectionActual','returnDirectionIntended','returnDirectionActual','missedTackler', 'assistTackler', 'tackler', 'kickoffReturnFormation', 'gunners', 'puntRushers', 'specialTeamsSafeties', 'vises', 'kickContactType', 'event', 'displayName', 'position', 'team', 'playDirection']:
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))
    
    train_x[c] = le.transform(train_x[c].fillna('NA'))
#   test_x[c] = le.transform(test_x[c].fillna('NA'))
    
train_x

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.1)

print("X_train", X_train.shape)
print("X_test", X_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)


In [ ]:
X_test.describe(include='all')


In [ ]:
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 80)
#X_train.dtypes

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

model = XGBRegressor( #目的関数の指定 初期値も二乗誤差です
                       objective='reg:squarederror',
                       #学習のラウンド数 early_stoppingを利用するので多めに指定　。。１０００回以上すべきですがテスト時間なく500回
                       n_estimators=500, 
                       #boosterに何を用いるか 初期値もgbtreeです
                       booster='gbtree',
                       #学習率
                       learning_rate=0.01,
                       #木の最大深さ
                       max_depth=6,
                       #シード値
                       random_state=2525)

#学習過程を表示するための変数を用意
evals_result = {}
model.fit(X_train, y_train,
        eval_set=[(X_train, y_train),(X_test, y_test)],
        #学習に用いる評価指標
        eval_metric='rmse',
        #目的関数の改善が見られない場合に学習を打ち切るラウンド数を指定。１０以上にすべきですがテスト時間なく５回
        early_stopping_rounds=5)
        #学習過程の記録をするためにコールバックAPIを利用し先ほどの変数を指定
#        callbacks=[xgb.callback.record_evaluation(evals_result)])

#model.fit(X_train, y_train,eval_set=[(X_train, y_train),(X_test, y_test)])

pred = model.predict(X_test)

#pred_label = np.where(pred > 0.5,1 ,0)

#submission = pd.DataFrame({'returnerID': test['returnerID'], 'Survived': pred_label})
#submission.to_csv('submission_first.csv', index=False)

rmse = sqrt(mean_squared_error(y_test, pred))
#print(mean_squared_error(y_test, pred))
print(rmse)




In [ ]:
import plotly.express as px

#棒グラフを作成
fig = px.histogram(pred)
#軸の設定等
#fig.update_layout(width=100,height=50,margin=dict(l=10,r=10,b=5,t=5),
#                 yaxis=dict(range=[0,10]),xaxis=dict(dtick=1, autorange=True),title=dict(font = dict(size=20)))

#データを並び替え
#fig = px.bar(scores)
fig.update_layout(xaxis={'categoryorder':'total descending'})
#グラフの表示
fig.show()

#Id = np.array(test["Id"]).astype(int)
result = pd.DataFrame(pred, columns = ["returnyard"])

# 予測データの確認
#result.head(10)
#print(y_test.head(10))

In [ ]:
plt.scatter(y_test, pred, alpha = 0.5)
plt.plot(np.linspace(0, 100, 100), np.linspace(0, 100, 100), "red")
plt.show()


In [ ]:
import xgboost as xgb
xgb.plot_importance(model)
plt.show()

In [ ]:
submission = pd.DataFrame({'returnerId': X_test['returnerId'], 'returnyard': pred})
submission.to_csv('submission_first.csv')


In [ ]:
X_test.to_csv("output_pd.csv")

In [ ]:
#!pip install shap
import shap

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(model=model, feature_perturbation='tree_path_dependent', model_output='margin')
shap_values = explainer.shap_values(X=X_train)

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X_train)

In [ ]:
shap_interaction_values = shap.TreeExplainer(model).shap_interaction_values(X_train)
shap.summary_plot(shap_interaction_values, X_train)

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader



In [ ]:
import os
from os.path import join

import numpy as np
import pandas as pd

from scipy.spatial import distance_matrix

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

try:
    import dgl
except:
    !pip install dgl
    import dgl
    
import warnings
warnings.filterwarnings('ignore')